In [8]:
import os
import glob
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='api.env',override=True)

True

In [3]:
openai_api_key=os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ali-onboarding-api-key')

In [4]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [12]:
!pip install --upgrade pip setuptools wheel

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 72.1.0
    Uninstalling setuptools-72.1.0:
      Successfully uninstalled setuptools-72.1.0 0/2 [setuptools]
  Attempting uninstall: pip━━━━━━━━━━━━━━━━━━━━━ 0/2 [setuptools]
    Found existing installation: pip 25.1━━━ 0/2 [setuptools]
    Uninstalling pip-25.1:━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [pip]
      Successfully uninstalled pip-25.1━━━━━━━━━━━━━━━━━━━ 1/2 [pip]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pip]1/2 [pip]


In [5]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
# Use DocArrayInMemorySearch instead of Chroma to avoid dependency conflicts
from langchain.vectorstores import DocArrayInMemorySearch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [6]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [28]:
import os, glob
ROOT = "/Users/apple/Downloads/My Files/My File/RAG/community contributions/Personal Knowledge Worker/Sameer-db"
folders = [p for p in glob.glob(os.path.join(ROOT, "*")) if os.path.isdir(p)]
print(folders)

['/Users/apple/Downloads/My Files/My File/RAG/community contributions/Personal Knowledge Worker/Sameer-db/personal', '/Users/apple/Downloads/My Files/My File/RAG/community contributions/Personal Knowledge Worker/Sameer-db/profile']


In [33]:
def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Created a chunk of size 1036, which is longer than the specified 1000


Total number of chunks: 10
Document types found: {'personal', 'profile'}


In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(documents=chunks, embedding=embeddings)
print(f"Vectorstore created with {len(chunks)} documents")

/opt/anaconda3/lib/python3.13/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Vectorstore created with 10 documents


In [46]:
# Let's investigate the vectors

# DocArrayInMemorySearch has a different API than ChromaDB
# We can get document count by doing a similarity search with a high k value
# and checking how many results we get back

# Get embedding dimensions by using the embeddings object directly
sample_embedding = embeddings.embed_query("sample text")
dimensions = len(sample_embedding)

# Try to get all documents by searching with a very high k value
# This will give us the total number of documents
try:
    all_results = vectorstore.similarity_search("", k=1000)  # High k to get all docs
    count = len(all_results)
    print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")
    
    # Show some sample documents
    if count > 0:
        sample_results = vectorstore.similarity_search("sample query", k=min(3, count))
        print(f"\nSample documents found:")
        for i, doc in enumerate(sample_results):
            print(f"{i+1}. {doc.page_content[:100]}...")
    else:
        print("No documents in the vector store")
        
except Exception as e:
    print(f"Error accessing vector store: {e}")
    print("Vector store may be empty or there's an issue with the search")


There are 10 vectors with 1,536 dimensions in the vector store

Sample documents found:
1. # Sameer Khadatkar

Hi, I am **Sameer Khadatkar**, born and brought up in **Nagpur**.

I completed m...
2. ---

## Certifications

- Advanced Data Science with IBM (Coursera)
- HYPERMESH (SHELL MESH AND SOLI...
3. ### 🧠 Word Embedding Explainability Research

- Collaborated with the Bank’s Chief Model Risk Office...


In [49]:
# Get all documents and their embeddings for visualization
# DocArrayInMemorySearch doesn't have a collection.get() method, so we'll use similarity_search

# Get all documents by searching with a high k value
all_results = vectorstore.similarity_search("", k=1000)

# Extract documents and metadata
documents = [doc.page_content for doc in all_results]
metadatas = [doc.metadata for doc in all_results]

# Get embeddings for each document
vectors = []
for doc in all_results:
    # Create embedding for each document
    embedding = embeddings.embed_query(doc.page_content)
    vectors.append(embedding)

vectors = np.array(vectors)

# Extract document types and create colors
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['green', 'red'][['personal', 'profile'].index(t)] for t in doc_types]

print(f"Retrieved {len(documents)} documents with {vectors.shape[1]} dimensional embeddings")

Retrieved 10 documents with 1536 dimensional embeddings


In [ ]:
# (t-distributed stochastic neighbor embedding)
tsne = TSNE(n_components=2, random_state=42,perplexity=5)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()


In [51]:
tsne = TSNE(n_components=3, random_state=42,perplexity=5)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()


In [52]:
from langchain.schema import SystemMessage

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# setting up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
memory.chat_memory.messages.insert(0, SystemMessage(
    content="""You are an AI Assistant specialized in providing accurate information about Sameer Khadatkar. Only respond when the question explicitly asks for information. 
    Keep your answers brief, factual, and based solely on the information provided. Do not speculate or fabricate details. 
    For example, if the user simply says "hi," respond with: "How can I help you?"
    """
))

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(k=4)

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/0q/0xxb80s51vg76dkx_k08zm2c0000gn/T/ipykernel_10824/3398974231.py:4: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [ ]:
query = "Who are you?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])
# setting up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

I am Sameer Khadatkar, a Quantitative Analytics Specialist at Wells Fargo in Hyderabad. I have a background in Mechanical Engineering and completed my M.Tech at the Indian Institute of Science, Bangalore. My expertise lies in validating AI/ML models, particularly in areas such as fraud detection, marketing, and credit scoring.


In [56]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [57]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
